# Activity Log Routes

> routes for interacting with the activity log


In [15]:
# | default_exp routes.activity_log

In [16]:
# | exporti

import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [17]:
# | export
@gd.route_function
async def get_activity_log_object_types(
    auth: dmda.DomoAuth,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    """retrieves a list of valid objectTypes that can be used to search the activity_log API"""

    url = f"https://{auth.domo+_instance}.domo.com/api/audit/v1/user-audits/objectTypes"

    return await gd.get_data(
        url=url,
        method="GET",
        auth=auth,
        parent_class=parent_class,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

#### sample implementation of get_activity_log_object_types


In [18]:
import os
import pandas as pd

try:
    token_auth = dmda.DomoTokenAuth(
        domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
        domo_instance="domo-community",
    )

    al_objecttype_res = await get_activity_log_object_types(auth=token_auth)

    print(pd.DataFrame(al_objecttype_res.response))

    # use to update `ActivityLog_ObjectType` enum
    # [ f"{objecttype.get('translation').upper().replace(' ', '_')} = '{objecttype.get('type')}'" for objecttype in  al_objecttype_res.response]

except Exception as e:
    print(e)

get_activity_log_object_types() got an unexpected keyword argument 'debug_num_stacks_to_drop'


In [20]:
# | export
@gd.route_function
async def search_activity_log(
    auth: dmda.DomoAuth,
    start_time: int,  # epoch time in milliseconds
    end_time: int,  # epoch time in milliseconds
    maximum: int,
    object_type: str = None,
    debug_api: bool = False,
    debug_loop: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    """loops over activity log api to retrieve audit logs"""

    is_close_session = False

    if not session:
        session = httpx.AsyncClient()
        is_close_session = True

    url = f"https://{auth.domo_instance}.domo.com/api/audit/v1/user-audits"

    if object_type and object_type != "ACTIVITY_LOG":
        url = f"{url}/objectTypes/{object_type}"

    fixed_params = {"end": end_time, "start": start_time}

    offset_params = {
        "offset": "offset",
        "limit": "limit",
    }

    def arr_fn(res) -> list[dict]:
        return res.response

    res = await gd.looper(
        auth=auth,
        method="GET",
        url=url,
        arr_fn=arr_fn,
        fixed_params=fixed_params,
        offset_params=offset_params,
        session=session,
        maximum=maximum,
        debug_loop=debug_loop,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if is_close_session:
        await session.aclose()

    return res

In [21]:
import os
import pandas as pd
import datetime as dt

import domolibrary.utils.convert as convert

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

page_res = await search_activity_log(
    object_type="PAGE",
    start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
    end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
    auth=token_auth,
    maximum=16,
    debug_loop=False,
    debug_api=False,
)

pd.DataFrame(page_res.response[0:5])

,objectId,objectName
0,-100000,Overview
1,-100001,Alerts
2,-100002,Shared
3,-100003,Favorites
4,-100012,CEO


#### sample implementation of search_activity_log


In [22]:
import os
import pandas as pd
import datetime as dt

import domolibrary.utils.convert as convert

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-community",
)

activity_log_res = await search_activity_log(
    object_type="ACTIVITY_LOG",
    start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
    end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
    auth=token_auth,
    maximum=10,
    debug_loop=False,
    debug_api=False,
)

pd.DataFrame(activity_log_res.response[0:5])

,userName,userId,userType,actorName,actorId,actionType,objectName,objectId,objectType,additionalComment,time,eventText,clientId,authenticationMethod
0,Jae Wilson1,1893952720,USER,,0,UPDATED,domolibrary test account - updated 2023-11-10,71,ACCOUNT,Jae Wilson1 updated Account domolibrary test a...,1699637687516,Updated Account,None,DEV_TOKEN
1,Jae Wilson1,1893952720,USER,,0,UPDATED,domo_creds,71,ACCOUNT,Jae Wilson1 updated Account domo_creds.,1699637687075,Updated Account,None,DEV_TOKEN
2,Ben Schein,1865099622,USER,,0,DOWNLOADED,Image background,505,FILE_REVISION,Ben Schein downloaded Image background from fi...,1699637678378,Downloaded a file version,None,JWT_TOKEN
3,Ben Schein,1865099622,USER,,0,DOWNLOADED,Image background,505,FILE_REVISION,Ben Schein downloaded Image background from fi...,1699637678158,Downloaded a file version,None,JWT_TOKEN
4,Ben Schein,1865099622,USER,,0,DOWNLOADED,Image background,505,FILE_REVISION,Ben Schein downloaded Image background from fi...,1699637678046,Downloaded a file version,None,JWT_TOKEN


In [25]:
# | hide
import nbdev

nbdev.nbdev_export()
!nbqa black activity_log.ipynb